# 260. Generative Adverserial Networks in PyTorch

- GAN은 훈련하기가 매우 어려울 수 있으며 hyperparameters, activation function 및 regularization 에 매우 민감  

- GAN을 훈련시켜 MNIST 와 비슷한 손으로 쓴 숫자의 이미지를 생성
<img src="https://i.imgur.com/CAYnuo1.jpg" width="360" >


- 다음의 순서로 작업

    * MNIST data load 
    * Discriminator network 정의
    * Generator network 정의
    * 손실함수 정의
    * Train the model
    * 중간 출력물 저장
    * model 저장

## Data Loading

- ``torchvision.datasets.MNIST``는 `[0, 1]` 사이로 scaling 되어 있음 

- 픽셀 값을`[0, 1]`범위에서`[-1, 1]`범위로 변환 
$$\frac{x-\mu}{\sigma}$$  
$$\mu : 0.5, \sigma : 0.5$$

- Generator 가 Tanh 함수를 activation 으로 이용하여 fake image 를 생성하기 때문에 Generator 의 출력 범위에 맞추어 준다.  

- image 가 `[-1, 1]`범위로 변환 되었는지 확인

- denormalized (0, 1 사이의 원래 값으로 변환) 하는 helper 함수를 정의 

### DataLoader 생성

## Discriminator Network 정의

- 28x28 이미지를 784 크기의 벡터로 취급  
- 3 hidden layer의 간단한 Sequential model 구축  
- 이진분류 모델 - 입력 vector가 real image 입력 이미지가 진짜일 확률 출력  (1-real, 0-fake)
- Leaky ReLU 활성 함수를 사용 (backprop 에서 0 의 기울기(경사)를 전달하는 대신 작은 음의 기울기를 전달)  

<img src="https://cdn-images-1.medium.com/max/1600/1*ypsvQH7kvtI2BhzR2eT_Sw.png" width="360">

## Generator Network 정의

- Generator 에 대한 입력은 이미지를 생성하기 위한 시드(seed)로 사용되는 latent vector  
- 3 개의 레이어로 구성된 신경망  
- 출력은 28x28 pixel 이미지로 변환될 수 있는 784 크기의 벡터  
- Generator 의 출력 레이어에 Tanh 활성화 함수를 사용  

<img src="https://nic.schraudolph.org/teach/NNcourse/figs/tanh.gif" width="240" >

## Discriminator(감별자) Training

- 감별자는 이진 분류 모델이므로 binary cross entropy 손실 함수를 사용  

### 감별자 훈련 단계

- image가 실제의 MNIST에서 제공되면 감별자는 1을 출력해야 한다.
- 생성자(Generator)가 생성한 image인 경우 0을 출력해야 한다.    

- 먼저 실제 이미지 배치를 전달하고 대상 레이블을 1로 설정하여 손실을 계산
- 그런 다음 generator 를 사용하여 가짜 이미지 배치를 생성하고 대상 레이블을 0으로 설정하여 descriminator 에 전달한 다음 손실을 계산  

- 두 손실을 더하고 전체 손실을 사용하여 Gradient Descent를 수행하여 Discriminator의 weight를 조정  

- 감별자를 훈련시키는 동안 생성자 모델의 가중치를 변경하지 않는다는 점에 유의 (`d_optimizer`는`D.parameters ()`에만 영향을 미침).

- gradient reset 및 discriminator train helper 함수 정의

## Generator Training

- 생성자의 출력은 이미지이므로 **손실 함수의 일부로 감별자를 사용**하는 트릭을 사용  

- 작동 방식은 다음과 같다.

- 우리는 생성자를 사용하여 이미지 배치를 생성하고 감별자에 전달.

- target 레이블을 1, 즉 real 로 설정하여 손실을 계산. (생성자의 목표는 감별자를 속이는 것이기 때문)

- 손실을 사용하여 경사 하강을 수행. 즉, 생성자의 weight 를 변경하므로 점점 더 실제 같은 이미지를 생성하게 된다.

## Training the Model

- 모델의 진행 상황을 육안으로 검사하기 위해 생성기의 중간 출력을 저장할 수있는 디렉토리를 생성

- 생성 된 이미지를 보면서 시각적 비교에 사용할 수있는 실제 이미지 배치를 저장

- 각 epoch 이 끝날 때 마다 생성된 이미지를 디스크에 저장하는 helper function 을 정의  

- 모델을 학습함에 따라 생성된 개별 이미지가 시간이 지남에 따라 어떻게 진화하는지 확인하기 위해 고정된 입력 벡터 세트를 generator 에 입력으로 제공

## Train 

- 각 epoch 마다 먼저 Discriminator 를 훈련시킨 다음 Generator 를 훈련시킨다.
- GPU 필요

모델을 학습 했으므로 checkpoint 를 저장할 수 있습니다.

10, 50, 100, 300 epoch 훈련 후 생성된 이미지가 어떻게 보이는지 보여줍니다.

- 처음에는 Discriminator 가 잘 구별 하므로 D 의 손실이 낮고 G 의 손실이 높지만 시간이 갈 수록 둘이 수렴하는 모습을 보여야 한다.

- D 가 초기에는 Real 과 Fake 를 잘 감별하므로 real_score 는 1 에 fake_score 는 0 에 가깝게 시작하지만 시간이 흐를 수록 real_score 가 떨어지고 fake_score 가 올라가서 수렴하는 모양이 되어야 한다.